Загрузка данных

In [90]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
import datetime

API_KEY = "PK1W6UGQYV6DJYD9HEP7"
SECRET_KEY = "ndH61mpUAGkK7jNKM8fgl1EvjUQKf0yDofyYP0pe"

client = CryptoHistoricalDataClient()
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD", 'ETH/USD', 'USDT/USD', 'XRP/USD', 'SOL/USD', 'USDC/USD', 'DOGE/USD', 'BCH/USD', 'LINK/USD', 'AVAX/USD'],
  timeframe=TimeFrame.Day,
  start=datetime.datetime(2020, 7, 1),
  end=datetime.datetime(2025, 7, 1)
)

btc_bars = client.get_crypto_bars(request_params)

df = btc_bars.df

Добавление меток о сезоне и дне недели

In [91]:
import pandas as pd

ts = df.index.get_level_values('timestamp')
ts = pd.to_datetime(ts)

dow_dummies = pd.get_dummies(ts.day_name(), prefix='is', dtype=bool).reindex(
    columns=['is_Monday', 'is_Tuesday', 'is_Wednesday', 'is_Thursday',
             'is_Friday', 'is_Saturday', 'is_Sunday'], fill_value=False)

def season(month):
    if month in (12, 1, 2):
        return 'winter'
    elif month in (3, 4, 5):
        return 'spring'
    elif month in (6, 7, 8):
        return 'summer'
    else:
        return 'autumn'

season_dummies = pd.get_dummies([season(m) for m in ts.month], prefix='is', dtype=bool).reindex(
    columns=['is_winter', 'is_spring', 'is_summer', 'is_autumn'], fill_value=False)

dow_dummies.index = df.index
season_dummies.index = df.index

df = df.join(dow_dummies).join(season_dummies)

Добавление индекса страха и жадности

In [92]:
import requests

def get_fear_greed_data():
    url = "https://api.alternative.me/fng/?limit=0&format=json"
    response = requests.get(url)
    data = response.json()['data']
    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'].astype(int), unit='s').dt.date
    df['fear&greed'] = df['value'].astype(int)
    return df[['timestamp', 'fear&greed']]

fng_df = get_fear_greed_data()

df = df.reset_index()
df['date'] = df['timestamp'].dt.date

df = df.merge(fng_df, how='left', left_on='date', right_on='timestamp')

df = df.drop(columns=['timestamp_y', 'date'])
df = df.rename(columns={'timestamp_x': 'timestamp'})

df = df.set_index(['symbol', 'timestamp'])

Подготовка к предобработке

In [93]:
import numpy as np

df = df.sort_index(level=["symbol", "timestamp"])

TRAIN_END = pd.Timestamp("2024-12-31", tz="UTC")
VAL_END   = pd.Timestamp("2025-04-30", tz="UTC")

FEATURE_COLS = [
    "open", "high", "low", "close", "volume",
    "trade_count", "vwap", "fear&greed"
]

Заполнение пропуска (NaN)

In [94]:
date_to_fix = pd.Timestamp('2024-10-26', tz='UTC')
ts_index = df.index.get_level_values('timestamp').normalize()
mask_date = ts_index == date_to_fix
df.loc[mask_date, 'fear&greed'] = df.loc[mask_date, 'fear&greed'].fillna(50)
df.isna().any().any()


False

Заполнение нулей в количественных признаках скользящим средним

In [95]:
num_cols = df.select_dtypes(include=['number']).columns
zero_mask = (df[num_cols] == 0).any(axis=1)
df_zeros = df[zero_mask]
df_zeros

open       high        low  \
symbol   timestamp                                                    
AVAX/USD 2023-06-28 05:00:00+00:00  12.977000  13.043369  12.091420   
         2023-07-15 05:00:00+00:00  14.480411  15.350190  14.435762   
         2023-07-16 05:00:00+00:00  14.596500  14.848036  14.158950   
         2023-07-18 05:00:00+00:00  14.288414  14.296072  13.609425   
         2023-07-20 05:00:00+00:00  14.064900  14.422093  13.681166   
...                                       ...        ...        ...   
USDT/USD 2024-06-21 05:00:00+00:00   0.999185   0.999405   0.998814   
         2024-07-13 05:00:00+00:00   1.000150   1.001000   0.999963   
         2024-07-20 05:00:00+00:00   1.000300   1.001000   1.000030   
         2024-09-02 05:00:00+00:00   0.999875   1.000288   0.999750   
         2024-09-08 05:00:00+00:00   0.999550   0.999650   0.999500   

                                        close  volume  trade_count  vwap  \
symbol   timestamp                                                         
AVAX/USD 2023-06-28 05:00:00+00:00  12.355550     0.0          0.0   0.0   
         2023-07-15 05:00:00+00:00  14.581827     0.0          0.0   0.0   
         2023-07-16 05:00:00+00:00  14.474308     0.0          0.0   0.0   
         2023-07-18 05:00:00+00:00  14.043191     0.0          0.0   0.0   
         2023-07-20 05:00:00+00:00  13.943046     0.0          0.0   0.0   
...                                       ...     ...          ...   ...   
USDT/USD 2024-06-21 05:00:00+00:00   0.999275     0.0          0.0   0.0   
         2024-07-13 05:00:00+00:00   1.000495     0.0          0.0   0.0   
         2024-07-20 05:00:00+00:00   1.000452     0.0          0.0   0.0   
         2024-09-02 05:00:00+00:00   1.000000     0.0          0.0   0.0   
         2024-09-08 05:00:00+00:00   0.999500     0.0          0.0   0.0   

                                    is_Monday  is_Tuesday  is_Wednesday  \
symbol   timestamp                                                        
AVAX/USD 2023-06-28 05:00:00+00:00      False       False          True   
         2023-07-15 05:00:00+00:00      False       False         False   
         2023-07-16 05:00:00+00:00      False       False         False   
         2023-07-18 05:00:00+00:00      False        True         False   
         2023-07-20 05:00:00+00:00      False       False         False   
...                                       ...         ...           ...   
USDT/USD 2024-06-21 05:00:00+00:00      False       False         False   
         2024-07-13 05:00:00+00:00      False       False         False   
         2024-07-20 05:00:00+00:00      False       False         False   
         2024-09-02 05:00:00+00:00       True       False         False   
         2024-09-08 05:00:00+00:00      False       False         False   

                                    is_Thursday  is_Friday  is_Saturday  \
symbol   timestamp                                                        
AVAX/USD 2023-06-28 05:00:00+00:00        False      False        False   
         2023-07-15 05:00:00+00:00        False      False         True   
         2023-07-16 05:00:00+00:00        False      False        False   
         2023-07-18 05:00:00+00:00        False      False        False   
         2023-07-20 05:00:00+00:00         True      False        False   
...                                         ...        ...          ...   
USDT/USD 2024-06-21 05:00:00+00:00        False       True        False   
         2024-07-13 05:00:00+00:00        False      False         True   
         2024-07-20 05:00:00+00:00        False      False         True   
         2024-09-02 05:00:00+00:00        False      False        False   
         2024-09-08 05:00:00+00:00        False      False        False   

                                    is_Sunday  is_winter  is_spring  \
symbol   timestamp                                                    
AVAX/USD 2023-06-28 05:00:00+00:00

In [96]:
for col in ['volume', 'trade_count', 'vwap']:
    df[col] = df[col].replace(0, np.nan)
    df[col] = df.groupby(level='symbol')[col].transform(
        lambda x: x.fillna(x.rolling(window=3, min_periods=1, center=True).mean())
    )


Заполнение пропусков в временных рядах

In [97]:
dfs_filled = []

for symbol, group in df.groupby(level='symbol'):
    group = group.sort_index(level='timestamp')

    # Получаем временную метку
    ts = group.index.get_level_values('timestamp')

    # Время первой записи (обычно 05:00 с timezone)
    t0 = ts[0].time()

    # Создаём полный индекс с нужным временем и timezone
    full_idx = (pd.date_range(start=ts.min().normalize(), end=ts.max().normalize(), freq='D', tz=ts[0].tz)
                + pd.Timedelta(hours=t0.hour, minutes=t0.minute))

    # Перепроиндексируем, чтобы добавить пропуски
    group = group.reset_index(level='symbol', drop=True)  # временный индекс только timestamp
    group = group.reindex(full_idx)

    # Интерполяция по времени
    group_interpolated = group.interpolate(method='time')

    # Восстанавливаем MultiIndex с именами уровней
    group_interpolated['symbol'] = symbol
    group_interpolated = group_interpolated.set_index('symbol', append=True)
    group_interpolated = group_interpolated.reorder_levels(['symbol', None])
    group_interpolated.index.set_names(['symbol', 'timestamp'], inplace=True)

    dfs_filled.append(group_interpolated)

# Объединяем обратно
df_filled = pd.concat(dfs_filled).sort_index()


C:\Users\haypr\AppData\Local\Temp\ipykernel_5148\3621683067.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group_interpolated = group.interpolate(method='time')
C:\Users\haypr\AppData\Local\Temp\ipykernel_5148\3621683067.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group_interpolated = group.interpolate(method='time')
C:\Users\haypr\AppData\Local\Temp\ipykernel_5148\3621683067.py:21: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group_interpolated = group.interpolate(method='time')
C:\Users\haypr\AppData\Local\Temp\ipykernel_5148\3621683067.py:21: FutureWarning: DataFrame.interpolate with object dtype

In [98]:
df = df_filled

Обрабатываем выбросы логарифмированием

In [99]:
for col in FEATURE_COLS:
    df[col] = np.log(df[col])

Делим выборку и масштабируем количественные признаки

In [100]:
from sklearn.preprocessing import StandardScaler

idx_ts = df.index.get_level_values("timestamp")

train = df[idx_ts <= TRAIN_END]
val   = df[(idx_ts > TRAIN_END) & (idx_ts <= VAL_END)]
test  = df[idx_ts > VAL_END]

feature_scalers = {}
train_scaled_list = []
val_scaled_list   = []
test_scaled_list  = []

for sym, g_train in train.groupby(level="symbol"):
    sc_feat = StandardScaler()
    sc_feat.fit(g_train[FEATURE_COLS])
    feature_scalers[sym] = sc_feat

    def scale_split(split_df):
        g = split_df.xs(sym, level="symbol", drop_level=False).copy()
        g[FEATURE_COLS] = sc_feat.transform(g[FEATURE_COLS])
        return g

    train_scaled_list.append(scale_split(train))
    val_scaled_list.append(scale_split(val))
    test_scaled_list.append(scale_split(test))


train_scaled = pd.concat(train_scaled_list).sort_index()
val_scaled   = pd.concat(val_scaled_list).sort_index()
test_scaled  = pd.concat(test_scaled_list).sort_index()


In [102]:
counts = df.groupby(level='symbol').size()

for symbol, count in counts.items():
    print(f"{symbol}: {count} записей")


AVAX/USD: 1321 записей
BCH/USD: 1642 записей
BTC/USD: 1642 записей
DOGE/USD: 1642 записей
ETH/USD: 1642 записей
LINK/USD: 1642 записей
SOL/USD: 1642 записей
USDC/USD: 683 записей
USDT/USD: 1642 записей
XRP/USD: 547 записей
